In [ ]:
pip install pandas numpy matplotlib scikit-learn tensorflow xgboost


In [ ]:
import os
import pandas as pd

# Define the folder containing the CSV files
folder_path = './stock_data'

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize a list to store individual DataFrames
dataframes = []

# Loop through the CSV files and read each one into a DataFrame
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all DataFrames into one single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_data.csv', index=False)

print("CSV files have been combined into combined_data.csv")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop

# ==========================
# Configuration Parameters
# ==========================
csv_path = 'combined_data.csv'  # Update with your CSV file path
target_col = 'Close'            # Update with the appropriate column name from your CSV
window_size = 30                # Number of time steps in each sample
debug_mode = False              # If True, use a small subset (e.g., 20 samples) to debug/overfit
learning_rate = 0.001
batch_size = 16
max_epochs = 100

# ==========================
# 1. Load and Preprocess Data
# ==========================
df = pd.read_csv(csv_path, parse_dates=['Date'])
print("Shape of df after reading CSV:", df.shape)
print("Columns in CSV:", df.columns.tolist())
print("Data sample:\n", df.head())

if df.empty:
    raise ValueError("The CSV file is empty. Please check the file content or path.")

df.sort_values('Date', inplace=True)
df.set_index('Date', inplace=True)

# Ensure target_col exists in the DataFrame
if target_col not in df.columns:
    raise KeyError(f"The column '{target_col}' was not found in the CSV file. Check header names.")

# ==========================
# 2. Create Labels Based on Data
# ==========================
# Compute change from previous row and create a binary label (1 for profit, 0 for loss/no gain)
df['Change'] = df[target_col] - df[target_col].shift(1)
df['Pct_Change'] = df[target_col].pct_change() * 100
df['Label'] = (df['Change'] > 0).astype(int)

# Only drop rows missing data in the target and 'Change' columns.
df.dropna(subset=[target_col, 'Change'], inplace=True)

print("After dropna, shape of df:", df.shape)
print("Label distribution:\n", df['Label'].value_counts())

# ==========================
# 3. Scale the Feature
# ==========================
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[[target_col]])
if scaled_data.shape[0] == 0:
    raise ValueError("Scaled data is empty. Check that the target column has data.")
df[f'{target_col}_Scaled'] = scaled_data

# ==========================
# 4. Create Sliding Window Dataset
# ==========================
def create_dataset(data, labels, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(labels[i+window_size])
    return np.array(X), np.array(y)

X_all, y_all = create_dataset(df[f'{target_col}_Scaled'].values, df['Label'].values, window_size)
print(f"Dataset shapes -> X: {X_all.shape}, y: {y_all.shape}")

if debug_mode:
    n_debug = min(20, len(X_all))
    X_all, y_all = X_all[:n_debug], y_all[:n_debug]
    print("Debug mode enabled: Using a small subset for training.")

# ==========================
# 5. Split the Data
# ==========================
# For time series, split chronologically: training on all but the most recent sample.
X_train, y_train = X_all[:-1], y_all[:-1]
X_test, y_test = X_all[-1:], y_all[-1:]
print(f"Training shapes -> X: {X_train.shape}, y: {y_train.shape}")
print(f"Test shapes -> X: {X_test.shape}, y: {y_test.shape}")

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test  = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# ==========================
# 6. Build the Model
# ==========================
model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(window_size, 1)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

optimizer = RMSprop(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ==========================
# 7. Callbacks and Training
# ==========================
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-6)

history = model.fit(
    X_train, y_train,
    epochs=max_epochs,
    batch_size=batch_size,
    verbose=1,
    callbacks=[early_stop, reduce_lr]
)

# ==========================
# 8. Evaluation and Prediction
# ==========================
prediction_prob = model.predict(X_test)[0, 0]
prediction_label = 1 if prediction_prob > 0.5 else 0

print(f"\nPrediction probability for profit on {df.index[-1].date()}: {prediction_prob:.2f}")
print("Predicted:", "Profit" if prediction_label == 1 else "Loss/No Gain")
print("Actual label:", "Profit" if y_test[0] == 1 else "Loss/No Gain")

# ==========================
# 9. Plot Training Metrics
# ==========================
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy', color='orange')
plt.title("Training Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
# --------------------------
# Prediction from Another File
# --------------------------
# Specify the path for the new CSV file to predict on.
new_csv_path = './stock_data/AAPL_data_with_sentiment.csv'  # Replace with your file path

# Load the new data
df_new = pd.read_csv(new_csv_path, parse_dates=['Date'])
print("Shape of new data:", df_new.shape)
print("New data sample:\n", df_new.head())

# Sort and set Date as index
df_new.sort_values('Date', inplace=True)
df_new.set_index('Date', inplace=True)

# IMPORTANT: Ensure the new data has the target column ("Close")
if target_col not in df_new.columns:
    raise KeyError(f"The column '{target_col}' was not found in the new CSV file.")

# Preprocess the new data
# If the new file has extra missing values in columns you care about, you might want to drop those rows.
# Here, we assume at least the target column exists and is valid.
df_new.dropna(subset=[target_col], inplace=True)

# Scale the target column using the same scaler that was fitted on training data.
# (Assuming 'scaler' is still in memory from the training code.)
df_new[f'{target_col}_Scaled'] = scaler.transform(df_new[[target_col]])

# Specify the prediction date in the new data
# (Make sure the date exists in the new file.)
prediction_date_str = "2024-11-10"  # Replace with the desired prediction date
prediction_date = pd.to_datetime(prediction_date_str)

# Select all data up to and including the prediction date
df_new_until_pred = df_new.loc[:prediction_date]

if len(df_new_until_pred) < window_size:
    raise ValueError(f"Not enough data to form a window of size {window_size} for prediction on {prediction_date_str}")

# Create the input sample: take the last 'window_size' rows from the new data
X_new = df_new_until_pred.tail(window_size)[f'{target_col}_Scaled'].values
X_new = X_new.reshape((1, window_size, 1))  # Reshape to 3D for LSTM

# Use the trained model to predict
new_pred_prob = model.predict(X_new)[0, 0]
new_pred_label = 1 if new_pred_prob > 0.5 else 0

print(f"\nPrediction for {prediction_date_str} from new file:")
print(f"Prediction probability for profit: {new_pred_prob:.2f}")
print("Predicted:", "Profit" if new_pred_label == 1 else "Loss/No Gain")
